# Huấn luyện mô hình NER với spaCy 3.x

## I. Import

In [27]:
import spacy
from spacy.tokens import DocBin
from spacy.training.example import Example
from spacy.training.iob_utils import offsets_to_biluo_tags
import random
import json

## II. Load Data

In [23]:
# Load dữ liệu
with open("./data/ner_data.json", "r", encoding="utf-8") as f:
    raw = json.load(f)["dataset"]

In [24]:
# Tạo mô hình trắng
nlp = spacy.blank("vi")
ner = nlp.add_pipe("ner")

In [25]:
# Bổ sung nhãn từ dữ liệu
for item in raw:
    for _, _, label in item["entities"]:
        ner.add_label(label)

In [28]:
# Chuẩn bị dữ liệu huấn luyện, loại bỏ entity sai alignment
TRAIN_DATA_CLEAN = []

for item in raw:
    text = item["text"]
    entities = item["entities"]
    doc = nlp.make_doc(text)
    try:
        tags = offsets_to_biluo_tags(doc, entities)
        if "-" not in tags:  # hợp lệ
            example = Example.from_dict(doc, {"entities": entities})
            TRAIN_DATA_CLEAN.append(example)
    except:
        continue  # bỏ qua nếu sai lớn hơn

print(f"Số mẫu hợp lệ: {len(TRAIN_DATA_CLEAN)}")

D:\Programs\Python\Python311\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Hệ Liên kết quốc tế-K7340201X của ngành Khoa học m..." with entities "[[3, 29, 'Programme'], [40, 59, 'Major']]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
D:\Programs\Python\Python311\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Hệ Tiên tiến: Luật (Chuyên ngành Luật kinh tế) của..." with entities "[[3, 46, 'Programme'], [57, 74, 'Major']]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
D:\Programs\Python\Python311\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be ali

Số mẫu hợp lệ: 9423


D:\Programs\Python\Python311\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Hệ Tiên tiến: Luật (Chuyên ngành Luật thương mại q..." with entities "[[3, 57, 'Programme'], [68, 89, 'Major']]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


## III. Training

In [29]:
# Huấn luyện mô hình
optimizer = nlp.begin_training()
for i in range(10):
    random.shuffle(TRAIN_DATA_CLEAN)
    losses = {}
    for example in TRAIN_DATA_CLEAN:
        nlp.update([example], drop=0.3, sgd=optimizer, losses=losses)
    print(f"Epoch {i+1} - Loss: {losses}")

Epoch 1 - Loss: {'ner': np.float32(1639.6969)}
Epoch 2 - Loss: {'ner': np.float32(300.68536)}
Epoch 3 - Loss: {'ner': np.float32(299.97)}
Epoch 4 - Loss: {'ner': np.float32(184.85022)}
Epoch 5 - Loss: {'ner': np.float32(135.07251)}
Epoch 6 - Loss: {'ner': np.float32(85.19305)}
Epoch 7 - Loss: {'ner': np.float32(135.1073)}
Epoch 8 - Loss: {'ner': np.float32(72.80031)}
Epoch 9 - Loss: {'ner': np.float32(148.09901)}
Epoch 10 - Loss: {'ner': np.float32(92.86531)}


## IV. Save model

In [30]:
# nlp.to_disk("../ner_model")
# print("Mô hình đã lưu vào thư mục ner_model")

# Lưu mô hình
nlp.to_disk("../ner_model_cleaned")
print("Đã lưu mô hình vào thư mục ner_model_cleaned")

Đã lưu mô hình vào thư mục ner_model_cleaned


## V. Save invalid NER data

In [ ]:
# with open("data/invalid_ner_data.json", "w", encoding="utf-8") as f:
#     json.dump(invalid_data, f, ensure_ascii=False, indent=2)
# print(f"Đã lưu {len(invalid_data)} câu lỗi vào file: data/invalid_ner_data.json")

Đã lưu 1120 câu lỗi vào file: data/invalid_ner_data.json


## VI. Test

In [31]:
ner_nlp = spacy.load("../ner_model_cleaned")

In [32]:
# ======== METHODS =========
def extract_entities(text: str):
    doc = ner_nlp(text)
    return [
        {
            "text": ent.text,
            "start": ent.start_char,
            "end": ent.end_char,
            "label": ent.label_
        }
        for ent in doc.ents
    ]

In [47]:
text = "Ngành quản trị du lịch có học phí bao nhiêu"
entities = extract_entities(text)

print(entities)

[{'text': 'quản trị du lịch', 'start': 6, 'end': 22, 'label': 'Major'}]
